Stable Diffusion Mega

The Stable Diffusion Mega Pipeline lets you use the main use cases of the stable diffusion pipeline in a single class. This script was contributed by [Patrick von Platen](https://github.com/patrickvonplaten/) and the notebook by [Parag Ekbote](https://github.com/ParagEkbote).



In [1]:
pip install torch diffusers 

Note: you may need to restart the kernel to use updated packages.


In [5]:
#!/usr/bin/env python3
from diffusers import DiffusionPipeline
import PIL
import requests
from io import BytesIO
import torch
import time

def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")

pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", custom_pipeline="stable_diffusion_mega", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")
pipe.enable_attention_slicing()

# Create a directory to save images
import os
output_dir = "generated_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Text-to-Image
images = pipe.text2img("An astronaut riding a horse").images
for idx, image in enumerate(images):
    image_path = os.path.join(output_dir, f"text2img_{time.time()}.png")
    image.save(image_path)
    print(f"Saved text-to-image output to {image_path}")

### Image-to-Image
init_image = download_image("https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg")
prompt = "A fantasy landscape, trending on artstation"
images = pipe.img2img(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images
for idx, image in enumerate(images):
    image_path = os.path.join(output_dir, f"img2img_{time.time()}.png")
    image.save(image_path)
    print(f"Saved image-to-image output to {image_path}")

### Inpainting
img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"
init_image = download_image(img_url).resize((512, 512))
mask_image = download_image(mask_url).resize((512, 512))
prompt = "a cat sitting on a bench"
images = pipe.inpaint(prompt=prompt, image=init_image, mask_image=mask_image, strength=0.75).images
for idx, image in enumerate(images):
    image_path = os.path.join(output_dir, f"inpaint_{time.time()}.png")
    image.save(image_path)
    print(f"Saved inpainting output to {image_path}")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Saved text-to-image output to generated_images/text2img_1731761960.293113.png


  0%|          | 0/37 [00:00<?, ?it/s]

Saved image-to-image output to generated_images/img2img_1731761972.032268.png


  0%|          | 0/37 [00:00<?, ?it/s]

Saved inpainting output to generated_images/inpaint_1731761976.5258489.png
